# Fahrradmessstellen Ergebnisse Radzählungen 2020

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#CSV Dateien einlesen
df1 = pd.read_csv('fahrradzaehlungenbonnjanuarbismai2020.csv', sep=';', header=None, low_memory=False).iloc[2:,] \
    .reset_index(drop=True)
df2 = pd.read_csv('fahrradzaehlungenbonnjulibisdezember2020_1.csv', sep=';', header=None, low_memory=False).iloc[2:,] \
    .reset_index(drop=True)

#Vorbereiten zum mergen
df2_zaehlstellen = df2.iloc[0].reset_index(drop=True).tolist()
df1.columns = df1.iloc[0]
df1 = df1.drop(df1.index[0])
df2.columns = df2.iloc[0]
df2 = df2.drop(df2.index[0])

#Merge und Summen entfernen
df = pd.merge(df1, df2, on=df2_zaehlstellen, how='outer')
df.set_index('Time', inplace=True)
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M')
df = df.drop(columns=['5.03 BN - Nordbrücke (Südseite)', '5.04 BN - Nordbrücke (Nordseite)', 
                      '5.05 BN - Südbrücke (Südseite)', '5.06 BN - Südbrücke (Nordseite)', 
                      '5.01 BN - Kennedybrücke (Nordseite)', '5.02 BN - Kennedybrücke (Südseite) Barometer', 
                      '5.10 BN - Bröhltalweg'])


column_mapping = {
    '5.07 BN - Estermannufer': 'Estermannufer',
    '5.08 BN - Von-Sandt-Ufer': 'Von-Sandt-Ufer',
    '5.09 BN - Rhenusallee': 'Rhenusallee',
    '5.11 BN - Brühler Straße': 'Brühler Straße',
    '5.14 BN - Mc Cloy Weg': 'Mc Cloy Weg',
    '5.15 BN - Weg auf Damm Neil': 'Weg auf Damm Neil',
    '5.13 BN - Wilhelm-Spiritus-Ufer': 'Wilhelm-Spiritus-Ufer'
}
order = ['Kennedybrücke', 'Nordbrücke', 'Südbrücke', 'Estermannufer', 'Von-Sandt-Ufer', 'Rhenusallee', 
         'Brühler Straße', 'Wilhelm-Spiritus-Ufer', 'Mc Cloy Weg', 'Weg auf Damm Neil']
df.rename(columns=column_mapping, inplace=True)
df = df[order]
df = df.applymap(lambda x: -1 if pd.isna(x) else x).astype(int)
df = df.resample('H').sum()
df

,Kennedybrücke,Nordbrücke,Südbrücke,Estermannufer,Von-Sandt-Ufer,Rhenusallee,Brühler Straße,Wilhelm-Spiritus-Ufer,Mc Cloy Weg,Weg auf Damm Neil
Time,,,,,,,,,,
2020-01-01 00:00:00,53,4,17,0,6,6,5,12,1,5
2020-01-01 01:00:00,77,6,15,0,2,9,13,8,2,6
2020-01-01 02:00:00,108,11,20,0,11,4,8,7,0,4
2020-01-01 03:00:00,104,4,9,0,5,5,7,3,2,6
2020-01-01 04:00:00,36,7,5,0,1,4,4,1,0,3
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,73,9,-1,1,20,14,8,15,-1,16
2020-12-31 20:00:00,49,3,-1,1,2,1,7,2,-1,2
2020-12-31 21:00:00,29,1,-1,2,6,3,6,5,-1,3


In [3]:
percentage_negative = ((df < 0).sum() / df.count()) * 100
percentage_negative

0
Kennedybrücke             0.000000
Nordbrücke                0.011384
Südbrücke                58.481330
Estermannufer             0.011384
Von-Sandt-Ufer            0.011384
Rhenusallee               0.011384
Brühler Straße            0.011384
Wilhelm-Spiritus-Ufer     0.011384
Mc Cloy Weg              12.431694
Weg auf Damm Neil         0.011384
dtype: float64

## Auszüge

In [8]:
kennedy2020 = df['Kennedybrücke']
bonn2020_tag = df.sum(axis=1).resample('D').sum()
bonn2020_monat = bonn2020_tag.resample('M').sum()
%store bonn2020_tag bonn2020_monat kennedy2020

Stored 'bonn2020_tag' (Series)
Stored 'bonn2020_monat' (Series)
Stored 'kennedy2020' (Series)


In [9]:
df.to_csv('df_Bonn2020.csv')
bonn2020_tag.to_csv('Bonn2020_täglich.csv')